In [1]:
from glob import glob
import librosa
import numpy as np
import pandas as pd
import sys
import time
import datetime
from tqdm import tqdm
import random
import os
import gc
import cv2

from sklearn.model_selection import StratifiedKFold

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam, AdamW, lr_scheduler
from torch.distributions import Uniform
from torch.utils.data import DataLoader, Dataset

from torchlibrosa.stft import Spectrogram, LogmelFilterBank
from torchlibrosa.augmentation import SpecAugmentation


from efficientnet_pytorch import EfficientNet
from torchvision.models import resnet34, resnet50

import sys
sys.path.append('..')
from libs import transform as tr
from libs import spectrogram as spec

import warnings
warnings.filterwarnings("ignore")

/home/yuigahama/anaconda3/lib/python3.8/site-packages/torchaudio/backend/utils.py:53: UserWarning: "sox" backend is being deprecated. The default backend will be changed to "sox_io" backend in 0.8.0 and "sox" backend will be removed in 0.9.0. Please migrate to "sox_io" backend. Please refer to https://github.com/pytorch/audio/issues/903 for the detail.
  warnings.warn(


In [2]:
train_tp = pd.read_csv('../../data/add_no_call.csv')
submission = pd.read_csv('../../data/sample_submission.csv')

pred_target = list(submission.columns)[1:]


SR = 48000
LENGTH = SR * 10

In [3]:
def set_seed(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  # type: ignore
    torch.backends.cudnn.deterministic = True  # type: ignore
    torch.backends.cudnn.benchmark = False  # type: ignore
set_seed(53)

In [4]:
def normalize_melspec(X: np.ndarray):
    eps = 1e-6
    mean = X.mean()
    X = X - mean
    std = X.std()
    Xstd = X / (std + eps)
    norm_min, norm_max = Xstd.min(), Xstd.max()
    if (norm_max - norm_min) > eps:
        V = Xstd
        V[V < norm_min] = norm_min
        V[V > norm_max] = norm_max
        V = 255 * (V - norm_min) / (norm_max - norm_min)
        V = V.astype(np.uint8)
    else:
        # Just zero
        V = np.zeros_like(Xstd, dtype=np.uint8)
    return V

In [5]:
class RfcxDataSet(Dataset):
    def __init__(self,
                 df:pd.DataFrame,
                 data_path:str,
    ):
        self.df =  df
        self.path = data_path
        self.img_size = 256
        
        self.transform = tr.Compose([
          tr.OneOf([
            tr.GaussianNoiseSNR(min_snr=10),
            tr.PinkNoiseSNR(min_snr=10)
          ]),
          #tr.PitchShift(max_steps=2, sr=SR),
          #tr.TimeStretch(),
          #tr.TimeShift(sr=SR),
          #tr.VolumeControl(mode="sine")
        ])
        
    
    def __len__(self):
        return len(self.df)
    
    def get_duration(self, t_min, t_max):
        annotated_duration = t_max - t_min
        
        if annotated_duration > 10:
            limit_sec = t_max - 10
            start_sec = random.randint(t_min, limit_sec)
            end_sec = start_sec + 10
            
            start = start_sec * SR
            end = end_sec * SR
        else:
            res_time = 10 - annotated_duration
            front_limit = res_time if res_time < t_min else t_min
            
            front_time = random.randint(0, front_limit)
            
            back_limit = 60 - t_max
            
            tmp_time = res_time - front_time
            back_time = tmp_time if tmp_time < back_limit else back_limit
            
            if not tmp_time < back_limit:
                front_time += tmp_time - back_limit
            
            start = (t_min - front_time) * SR
            end = (t_max + back_time) * SR
            
        return start, end
    
    def get_no_call(self, t_min, t_max):
        t_min = int(t_min * 0.9)
        t_max = int(t_max * 1.1)
        
        back_time = 60 - t_max
        
        if t_min > back_time:
            limit = t_min - 11
            start = random.randint(0, limit)
        else:
            limit = 50
            start = random.randint(t_max+1, limit)
        end = start + 10
        
        return start * SR, end * SR
    
    def __getitem__(self, idx: int):
        sample = self.df.iloc[idx, :]
        recording_id = sample['recording_id']
        species_id = sample['species_id']
        is_no_call = sample['is_nocall']
        t_min = int(round(sample['t_min']))
        t_max = int(round(sample['t_max']))
        
        record_path = self.path + recording_id + '.flac'
        y, sr = librosa.load(record_path, sr=None)
        
        random_limit = y.shape[0] - LENGTH
        
        start = random.randint(0, random_limit)
        end = start + LENGTH
        
        target = torch.zeros([25], dtype=torch.float32)
        
        if is_no_call:
            start, end = self.get_no_call(t_min, t_max)
            target[-1] = 1
        else:
            start, end = self.get_duration(t_min, t_max)
            target[species_id] = 1

        tarnsform_flag = random.choice([True, True, False])
        
        if tarnsform_flag:
            y = self.transform(y)
        
        
        return y[start:end], target

In [6]:
class RfcxTestDataSet(Dataset):
    def __init__(self,
                 df:pd.DataFrame,
                 data_path:str,
                 val=False
    ):
        self.df =  df
        self.val = val
        self.path = data_path
        self.img_size = 256
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx: int):
        sample = self.df.iloc[idx, :]
        recording_id = sample['recording_id']
        record_path = self.path + recording_id + '.flac'
        
        y, sr = librosa.load(record_path, sr=None)
        
        images = []
        
        start = 0
        for i in range(6):
            end = start + (SR * 10)
            images.append(y[start:end])
            start = end
            
        if self.val == True:
            species_id = sample['species_id']
            target = torch.zeros([25], dtype=torch.float32)
            target[species_id] = 1
            return images, target
        else:
            return images

In [7]:
class BaseModel(nn.Module):
    def __init__(self, model_type, model_name, output_size, spectrogram_params, logmel_extractor_params, spec_augmenter_params, pce_params):
        super().__init__()
        self.model_type = model_type
        
        self.spectrogram_extractor = Spectrogram(**spectrogram_params)

        # Logmel feature extractor
        self.logmel_extractor = LogmelFilterBank(**logmel_extractor_params)

        # Spec augmenter
        self.spec_augmenter = SpecAugmentation(**spec_augmenter_params)
        
        #Pcen converter
        self.pcen_converter = spec.pcen(**pce_params)
        
        self.bn0 = nn.BatchNorm2d(logmel_extractor_params['n_mels'])
        
        self.mixup_alpha = 0.2
        self.random_state = np.random.RandomState(123)
        
        if model_type == 'res':
            if model_name == 'resnet50':
                base_model = torch.hub.load('zhanghang1989/ResNeSt', 'resnest50', pretrained=True)
            else:
                base_model = resnet34(pretrained=True)
            
            layers = list(base_model.children())[:-1]
            self.encoder = nn.Sequential(*layers)

            in_features = base_model.fc.in_features

            #self.fc1 = nn.Linear(in_features, in_features, bias=True)
            
            self.fc1 = nn.Sequential(
                nn.Linear(in_features, 1024),
                nn.ReLU(),
                nn.Dropout(p=0.2),
                nn.Linear(1024, 1024),
                nn.ReLU(),
                nn.Dropout(p=0.2),
                nn.Linear(1024, output_size)
            )

            
        elif model_type == 'efficientnet':
            self.features = EfficientNet.from_pretrained(model_name)
            model_code = model_name.split('-')[1]
            
            num_input = {
                'b4': 1792,
                'b2': 1408,
                'b7': 2560
            }
            
            self.features_out = num_input[model_code]
            
            self.classification = nn.Sequential(nn.Linear(num_input[model_code], output_size))
            
    def mixup(self, x):
        lam = self.random_state.beta(self.mixup_alpha, self.mixup_alpha, 1)[0]
        index = list(range(x.size(0)))
        random.shuffle(index)
        out = (x * lam + x[index].squeeze() * (1-lam))
        
        return out, {'lam': lam, 'index': index}

    def norm_spec(self, x):
        batch_size = x.size(0)
        height = x.size(2)
        width = x.size(3)
        
        x = x.squeeze().view(batch_size, -1)
        
        x -= x.min(1, keepdim=True)[0]
        x /= x.max(1, keepdim=True)[0]
        x = x.view(batch_size, height, width)
        
        return x.unsqueeze(1)
        
    def forward(self, input):
        if self.model_type == 'res':
            x = self.spectrogram_extractor(input)
            x = self.logmel_extractor(x)
            
            x_mels = self.norm_spec(x)
            x_pcen = self.norm_spec(self.pcen_converter(x))
            x_clear = self.norm_spec(self.logmel_extractor.power_to_db(x ** 1.5))
            
            #x = torch.stack([self.norm_spec(x_mels), x_pcen, x_clear], dim=1).squeeze()
            x = torch.stack([x_mels, x_pcen, x_clear], dim=1).squeeze()
            
            x = x.transpose(1, 3)
            x = self.bn0(x)
            x = x.transpose(1, 3)
            
            if self.training:
                x, mix_info = self.mixup(x)
                x = self.spec_augmenter(x)
            else:
                mix_info = None
            
            x = self.encoder(x)

            x = F.dropout(x, p=0.5, training=self.training)
            x = self.fc1(x)
                        
        elif self.model_type == 'efficientnet':
            x = self.features.extract_features(input)
            x = F.avg_pool2d(x, x.size()[2:]).reshape(-1, self.features_out)
            x = self.classification(x)
            
        return x, mix_info

In [8]:
def save(fold, model, optim, criterion, file_path="../../model/"):
    if not TEST_NAME in os.listdir(file_path):
        os.mkdir(file_path+TEST_NAME)
    
    
    output_path = file_path + TEST_NAME + '/' + f"{TEST_NAME}_{fold}.model"
    
    torch.save(
        {
            'epoch': epoch,
            'model_state_dict': model.cpu().state_dict(),
            'optimizer_state_dict': optim.state_dict(),
            'criterion': criterion
        },
        output_path)
    
    model.to(device)
    
    return output_path

In [9]:
# LRAP. Instance-level average
# Assume float preds [BxC], labels [BxC] of 0 or 1
def LRAP(preds, labels):
    # Ranks of the predictions
    ranked_classes = torch.argsort(preds, dim=-1, descending=True)
    # i, j corresponds to rank of prediction in row i
    class_ranks = torch.zeros_like(ranked_classes)
    for i in range(ranked_classes.size(0)):
        for j in range(ranked_classes.size(1)):
            class_ranks[i, ranked_classes[i][j]] = j + 1
    # Mask out to only use the ranks of relevant GT labels
    ground_truth_ranks = class_ranks * labels + (1e6) * (1 - labels)
    # All the GT ranks are in front now
    sorted_ground_truth_ranks, _ = torch.sort(ground_truth_ranks, dim=-1, descending=False)
    pos_matrix = torch.tensor(np.array([i+1 for i in range(labels.size(-1))])).unsqueeze(0)
    score_matrix = pos_matrix / sorted_ground_truth_ranks
    score_mask_matrix, _ = torch.sort(labels, dim=-1, descending=True)
    scores = score_matrix * score_mask_matrix
    score = (scores.sum(-1) / labels.sum(-1)).mean()
    return score.item()

# label-level average
# Assume float preds [BxC], labels [BxC] of 0 or 1
def LWLRAP(preds, labels):
    # Ranks of the predictions
    ranked_classes = torch.argsort(preds, dim=-1, descending=True)
    # i, j corresponds to rank of prediction in row i
    class_ranks = torch.zeros_like(ranked_classes)
    for i in range(ranked_classes.size(0)):
        for j in range(ranked_classes.size(1)):
            class_ranks[i, ranked_classes[i][j]] = j + 1
    # Mask out to only use the ranks of relevant GT labels
    ground_truth_ranks = class_ranks * labels + (1e6) * (1 - labels)
    # All the GT ranks are in front now
    sorted_ground_truth_ranks, _ = torch.sort(ground_truth_ranks, dim=-1, descending=False)
    # Number of GT labels per instance
    num_labels = labels.sum(-1)
    pos_matrix = torch.tensor(np.array([i+1 for i in range(labels.size(-1))])).unsqueeze(0)
    score_matrix = pos_matrix / sorted_ground_truth_ranks
    score_mask_matrix, _ = torch.sort(labels, dim=-1, descending=True)
    scores = score_matrix * score_mask_matrix
    score = scores.sum() / labels.sum()
    return score.item()

def mixup_socre(cor, x, y, mix_info):
    return cor(x, y) * mix_info['lam'] + cor(x, y[mix_info['index']].squeeze()) * (1-mix_info['lam'])

# Sample usage
# y_true = torch.tensor(np.array([[1, 1, 0], [1, 0, 1], [0, 0, 1]]))
# y_score = torch.tensor(np.random.randn(3, 3))
# print(LRAP(y_score, y_true), LWLRAP(y_score, y_true))

In [10]:
TEST_NAME = 'baseline-pic-res'

n_splits = 5
random_state = 1
epochs = 50

skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

spectrogram_params = {
    'n_fft': 2048,
    'hop_length': 512,
    'win_length': 1024,
    'window': 'hann',
    'center': True,
    'pad_mode': 'reflect',
    'freeze_parameters': True
}

logmel_extractor_params = {
    'sr': SR,
    'n_fft': 2048,
    'n_mels': 80,
    'fmin': 0,
    'fmax': 14000,
    'ref' : 1.0,
    'amin': 1e-10,
    'top_db': None,
    'is_log': False,
    'freeze_parameters': True
}

spec_augmenter_params = {
    'time_drop_width':  64,
    'time_stripes_num': 2,
    'freq_drop_width':  8,
    'freq_stripes_num': 2
}

pce_params = {
    'gain': 0.98,
    'bias': 2,
    'power': 0.5,
    'time_constant': 0.4,
    'eps': 0.000001,
}

model_params = {
    'model_type': 'res', # res or efficientnet
    'model_name': 'resnet50',
    'output_size': 25,
    'spectrogram_params': spectrogram_params,
    'logmel_extractor_params': logmel_extractor_params,
    'spec_augmenter_params': spec_augmenter_params,
    'pce_params': pce_params
}

optim_params = {
    'lr': 1e-3,
    'weight_decay': 5e-5,
    'betas': (0.9, 0.999)
}

scheduler_params = {
    'mode': 'max',
    'patience': 1,
    'factor': 0.6,
    'verbose': False
}

data_params = {
    'data_path': '/home/yuigahama/kaggle/rfcx/data/train/',
}

test_data_params = {
    'data_path': '/home/yuigahama/kaggle/rfcx/data/test/',
}

dataloder_params = {
    'batch_size': 64,
    'num_workers': 15,
    'pin_memory': False,
}

test_dataloder_params = {
    'batch_size': 32,
    'num_workers': 15,
    'pin_memory': False,
    'shuffle':False
}

In [11]:
tta = np.zeros((len(submission), 24))
cv_score = 0

for fold_id, (train_index, val_index) in enumerate(skf.split(train_tp, train_tp.species_id)):
    print(f'---------- fold {fold_id} ----------')
    
    model = BaseModel(**model_params).to(device)
    optim = Adam(model.parameters(), **optim_params)
    scheduler = lr_scheduler.ReduceLROnPlateau(optimizer=optim, **scheduler_params)
    #scheduler = lr_scheduler.CosineAnnealingLR(optimizer=optim, T_max=epochs)
    
    pos_weights = torch.ones(25)
    pos_weights = pos_weights * 25
    criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weights)
    
    train_dataset = RfcxDataSet(train_tp.iloc[train_index], **data_params)
    val_dataset   = RfcxTestDataSet(train_tp.iloc[val_index], val=True, **data_params)
    test_dataset  = RfcxTestDataSet(submission, **test_data_params)

    train_dataloader = DataLoader(train_dataset, shuffle=True, **dataloder_params)
    val_dataloader = DataLoader(val_dataset, shuffle=False, **dataloder_params)
    test_dataloader = DataLoader(test_dataset, **test_dataloder_params)
    
    es = 15
    for epoch in range(1, epochs):
        if es == 0:
            break
        
        start_time = time.time()
        bast_score = 0
        
        # train
        model.train()
        train_loss = 0
        train_score = 0
        
        for data in train_dataloader:
            image = data[0].to(device)
            label = data[1].to(device)
            
            pred, mix_info = model(image)
            
            optim.zero_grad()
            
            loss = mixup_socre(criterion, pred.cpu(), label.cpu(), mix_info)
            score = mixup_socre(LWLRAP, torch.sigmoid(pred.cpu()), label.cpu(), mix_info)
            #loss = criterion(pred, label)
            #score = LWLRAP(pred.cpu(), label.cpu())
            
            loss.backward()
            optim.step()
            
            train_loss += loss.item()
            train_score += score
            
        train_loss  /= len(train_dataloader)
        train_score /= len(train_dataloader)
        
        # val
        model.eval()
        val_loss = 0
        val_score = 0
        
        with torch.no_grad():
            for val_data in val_dataloader:
                num = len(val_data[0])
                label = val_data[1]
                tmp = torch.zeros((val_data[0][0].size(0), 25))
                
                for img in val_data[0]:
                    pred, _ = model(img.to(device))
                    tmp += pred.cpu() / num
                
                val_loss += criterion(tmp, label).item()
                val_score += LWLRAP(torch.sigmoid(tmp), label)

                
        val_loss  /= len(val_dataloader)
        val_score /= len(val_dataloader)
        
        duration = str(datetime.timedelta(seconds=time.time() - start_time))[:7]
        print(f'epoch {epoch:3}| T | loss: {train_loss:.3} | score: {train_score:.3} | V | loss: {val_loss:.3} | score: {val_score:.3} | time: {duration}')

        if bast_score < val_score:
            bast_score = val_score
            bast_path = save(fold_id, model, optim, criterion)
        else:
            es -= 1
        
        scheduler.step(val_score)
            
        
    oof = np.zeros((len(test_dataset), 24))
    model = BaseModel(**model_params).to(device)
    
    bast_model_parms = torch.load(bast_path)
    model.load_state_dict(bast_model_parms['model_state_dict'])

    
    with torch.no_grad():
        start = 0
        for i, test_data in enumerate(test_dataloader):
            target_size = test_data[0].size(0)
            tmp_pred = np.zeros((target_size, 24))
            num = len(test_data)
            end = start + target_size
            for img in test_data:
                img = img.to(device)
                pred, _ = model(img)
                
                tmp_pred += torch.sigmoid(pred.cpu().detach()).numpy()[:, :-1] / num
                       
            oof[start:end, :] = tmp_pred
            start = end
            
    tta += (oof / n_splits) 
    
    del model, train_dataset, val_dataset, test_dataset, train_dataloader, val_dataloader, test_dataloader
    del bast_model_parms, optim
    gc.collect()

---------- fold 0 ----------


Using cache found in /home/yuigahama/.cache/torch/hub/zhanghang1989_ResNeSt_master


epoch   1| T | loss: 1.33 | score: 0.206 | V | loss: 1.26 | score: 0.272 | time: 0:00:35
epoch   2| T | loss: 1.22 | score: 0.277 | V | loss: 2.17 | score: 0.203 | time: 0:00:35
epoch   3| T | loss: 1.15 | score: 0.329 | V | loss: 1.48 | score: 0.289 | time: 0:00:36
epoch   4| T | loss: 1.08 | score: 0.378 | V | loss: 0.947 | score: 0.463 | time: 0:00:36
epoch   5| T | loss: 1.01 | score: 0.429 | V | loss: 1.02 | score: 0.41 | time: 0:00:36
epoch   6| T | loss: 0.928 | score: 0.481 | V | loss: 1.05 | score: 0.435 | time: 0:00:35
epoch   7| T | loss: 0.862 | score: 0.53 | V | loss: 0.933 | score: 0.489 | time: 0:00:37
epoch   8| T | loss: 0.835 | score: 0.556 | V | loss: 0.803 | score: 0.548 | time: 0:00:36
epoch   9| T | loss: 0.776 | score: 0.605 | V | loss: 0.894 | score: 0.535 | time: 0:00:37
epoch  10| T | loss: 0.802 | score: 0.602 | V | loss: 0.835 | score: 0.528 | time: 0:00:36
epoch  11| T | loss: 0.833 | score: 0.587 | V | loss: 0.712 | score: 0.605 | time: 0:00:37
epoch  12| 

Using cache found in /home/yuigahama/.cache/torch/hub/zhanghang1989_ResNeSt_master


---------- fold 1 ----------


Using cache found in /home/yuigahama/.cache/torch/hub/zhanghang1989_ResNeSt_master


KeyboardInterrupt: 

Process Process-1498:
Traceback (most recent call last):
  File "/home/yuigahama/anaconda3/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/yuigahama/anaconda3/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/home/yuigahama/anaconda3/lib/python3.8/site-packages/torch/utils/data/_utils/worker.py", line 216, in _worker_loop
    pass
KeyboardInterrupt


In [ ]:
print(cv_score)

In [ ]:
pred_sub = pd.DataFrame(tta, columns=pred_target)
pred_sub

In [ ]:
pd.concat([submission['recording_id'], pred_sub], axis=1).to_csv('submission.csv', index=False)